# Lab 2 - Memory Process Analysis
## Justin Vermillion
### 3 November 2025

First take a hash of the dump file. Normally would compare this with the hash from the person who gathered the dump to ensure no tampering, but one was not provided with this file.

In [1]:
! md5sum iveep/CTICOE-iveep_374_2025-03-17T17-30Z.mem

59526a220e839ac8139fec04539baeac  iveep/CTICOE-iveep_374_2025-03-17T17-30Z.mem


First step is to see which processes were running, using the pslist plugin in volatility3.

In [2]:
! vol -f iveep/CTICOE-iveep_374_2025-03-17T17-30Z.mem windows.pslist

Volatility 3 Framework 2.26.2
Progress:   19.58		Reading Symbol layer                                                                                               

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Progress:   85.52		Reading Symbol layer                                                                                               

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Progress:  100.00		PDB scanning finished                                                                                              
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

4	0	System	0x840e90261040	213	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
108	4	Registry	0x840e9037c040	4	-	N/A	False	2025-03-12 16:39:25.000000 UTC	N/A	Disabled
536	4	smss.exe	0x840e910b4040	2	-	N/A	False	2025-03-12 16:39:40.000000 UTC	N/A	Disabled
804	796	csrss.exe	0x840e921650c0	12	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
880	796	wininit.exe	0x840e9228c080	1	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
888	872	csrss.exe	0x840e92291140	10	-	1	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
980	872	winlogon.exe	0x840e92305080	3	-	1	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
1000	880	services.exe	0x840e9230b100	7	-	0	False	2025-03-12 16:39:52.000000 UTC	N/A	Disabled
548	880	lsass.exe	0x840e92349080	10	-	0	False	202

A couple of items that stand out to me right away are:
- PSEXESVC.exe: the remote execution service for PsExec
- excel-update.exe: this is a fishy filename that sounds like a fake updater

Next I will check the relationships between processes with pstree.

In [3]:
! vol -f iveep/CTICOE-iveep_374_2025-03-17T17-30Z.mem windows.pstree | cut -d$'\t' -f1,2,3

Volatility 3 Framework 2.26.2ng finished                        

PID	PPID	ImageFileName

4	0	System
* 536	4	smss.exe
* 1924	4	MemCompression
* 108	4	Registry
804	796	csrss.exe
880	796	wininit.exe
* 1000	880	services.exe
** 11776	1000	svchost.exe
** 3604	1000	svchost.exe
** 4636	1000	svchost.exe
** 11296	1000	PSEXESVC.exe
*** 4672	11296	excel-update.e
**** 13752	4672	cmd.exe
***** 12392	13752	conhost.exe
***** 13156	13752	powershell.exe
** 1580	1000	svchost.exe
*** 15784	1580	taskhostw.exe
*** 6380	1580	taskhostw.exe
*** 4428	1580	taskhostw.exe
*** 10476	1580	taskhostw.exe
** 2092	1000	svchost.exe
** 1076	1000	svchost.exe
** 3636	1000	svchost.exe
** 7220	1000	svchost.exe
** 4156	1000	svchost.exe
** 3648	1000	CSFalconServic
*** 6624	3648	CSFalconContai
*** 324	3648	CSFalconContai
*** 1192	3648	CSFalconContai
*** 6604	3648	CSFalconContai
*** 5980	3648	CSFalconContai
** 1604	1000	svchost.exe
** 8772	1000	svchost.exe
** 1612	1000	svchost.exe
** 604	1000	svchost.exe
** 1124	1000	svchost.exe

We can see that excel-update.exe was started by PSEXESVC.exe, which indicates it was executed remotely. It then spawned a cmd prompt which was used to spawn a powershell session. Seems suspicious to me.

- 11296	1000	PSEXESVC.exe
    - 4672	11296	excel-update.exe
        - 13752	4672	cmd.exe
            - 12392	13752	conhost.exe
            - 13156	13752	powershell.exe

It also looks like there was a spoolsv.exe spawned by a powershell.exe, which is pretty suspicious since I would expect the print spooler would've been spawned by services.exe (and it does appear to be, earlier in the tree).

- 9940	2804	userinit.exe
    - 9852	9940	explorer.exe
        - 15172	9852	powershell.exe
            - 13176	15172	conhost.exe
            - 14536	15172	spoolsv.exe